# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.
Notice that `PlaneWaveBasis` defaults to just using the ``Γ``-point
if no ``k``-point options are provided.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5)

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.125030e+02     1.657582e+00
 * time: 0.09037017822265625
     1     1.094351e+01     8.894407e-01
 * time: 1.1222901344299316
     2    -1.123011e+01     1.065517e+00
 * time: 1.456705093383789
     3    -3.400206e+01     8.288302e-01
 * time: 1.6057381629943848
     4    -4.748051e+01     6.912622e-01
 * time: 1.7523090839385986
     5    -5.683221e+01     2.541606e-01
 * time: 1.8968491554260254
     6    -5.970073e+01     2.109213e-01
 * time: 2.0211021900177
     7    -6.086458e+01     8.052445e-02
 * time: 2.133707046508789
     8    -6.134658e+01     4.866385e-02
 * time: 2.2382972240448
     9    -6.161708e+01     6.542788e-02
 * time: 2.3498101234436035
    10    -6.181331e+01     3.174593e-02
 * time: 2.4620699882507324
    11    -6.196156e+01     2.355427e-02
 * time: 2.5733799934387207
    12    -6.205066e+01     1.731614e-02
 * time: 2.6791491508483887
    13    -6.209438e+01     1.733940e-02
 * time: 2.7915871143341064


In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671089
    AtomicLocal         -18.8557738
    AtomicNonlocal      14.8522668
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485400 
    Xc                  -19.3336831

    total               -62.261666460146
